In [1]:
import numpy as np
sgm = lambda x: 1/(1+np.exp(-x))
nrand = np.random.random
zerov = np.zeros

In [74]:
class RBM:
    '''A restricted Boltzmann machine model.'''
    def __init__(self, n_variables, n_hidden, spin = False):
        sigma = np.sqrt(4./(n_hidden+n_variables))
        self.a = sigma*(2*nrand(n_variables)-1)
        self.b = zerov(n_hidden)
        self.w = sigma*(2*nrand((n_variables,n_hidden))-1)
        self.min = -1*spin # Evaluates to zero if the model is not spin-based

    def activate(self, v, w, b, gap = 1):
        '''Turns the neurons of a forward layer on, according to a sigmoid probability function.'''
        vec = v@w + b # The neurons evaluation
        l = len(vec) # sample size
        prob = sgm(gap*vec)
        out = np.full(prob.shape,self.min)
        out[prob < nrand(prob.shape)] = 1
        return out
    
    def sample(self, v_data, steps = 1):
        h_data = self.activate(v_data, self.w, self.b)
        h_model = h_data
        for _ in range(steps):
            v_model = self.activate(h_model, self.w.T, self.a)
            h_model = self.activate(v_model, self.w, self.b)
        return h_data, v_model, h_model
    
    def grad(self, v_data, h_data, v_model, h_model):
        da = np.average(v_data - v_model, axis = 0)
        db = np.average(h_data - h_model, axis = 0)
        dw = sum(np.outer(v_data[i],h_data[i])-np.outer(v_model[i],h_model[i]) for i in range(len(v_data)))/len(v_data)
        #np.average(np.outer(v_data,h_data)-np.outer(v_model,h_model), axis = 1)
        return da, db, dw

    def train(self, data, n_epochs = 10, batch_size = 30, learning_rate = 1):
        batches = len(data)//batch_size
        for epoch in range(n_epochs):
            np.random.shuffle(data)
            for i in range(batches):
                v = data[i*batch_size:(i+1)*batch_size]
                h, v_m, h_m = self.sample(v)
                ca, cb, cw = self.grad(v,h,v_m,h_m)
                self.a += learning_rate*ca
                self.b += learning_rate*cb
                self.w += learning_rate*cw
            learning_rate = learning_rate/(0.05*learning_rate+1)



In [101]:
data = np.loadtxt('DATA/dataRBM_q0.1.csv', delimiter = ',')
rbm = RBM(len(data[0]),12)
rbm.train(data, n_epochs = 30, batch_size = 20)

<ipython-input-1-1887cda36eef>:2: RuntimeWarning: overflow encountered in exp
  sgm = lambda x: 1/(1+np.exp(-x))


In [115]:
np.random.shuffle(data)
_, v_fantasy, h_fantasy = rbm.sample(data,1)

<ipython-input-1-1887cda36eef>:2: RuntimeWarning: overflow encountered in exp
  sgm = lambda x: 1/(1+np.exp(-x))


In [116]:
v_fantasy

array([[1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0]])

In [105]:
rbm.w

array([[-4981.92247684, -4981.84692478, -4981.29519853, -4981.61664486,
        -4981.62154184, -4981.02823632, -4982.09367367, -4981.39069301,
        -4981.97611824, -4981.71063018, -4981.70934039, -4981.7279874 ],
       [ 3971.09726579,  3971.1377993 ,  3970.71347986,  3970.72847893,
         3970.71215986,  3971.00089361,  3970.74441765,  3970.86394419,
         3970.52381895,  3971.27931158,  3971.11814402,  3970.60198327],
       [-4830.16822945, -4829.89676082, -4829.43820657, -4829.53109636,
        -4829.9867077 , -4829.84178521, -4830.46846492, -4830.00231587,
        -4830.29851532, -4829.37832879, -4829.55256813, -4829.95971475],
       [ 4148.82812424,  4148.27376499,  4147.477894  ,  4148.63894966,
         4148.33279468,  4148.39967432,  4148.54235842,  4148.75895918,
         4148.68034508,  4148.17605616,  4148.41685153,  4148.53654229],
       [-4679.15318588, -4679.08262061, -4678.79764581, -4679.48195354,
        -4679.33022186, -4678.64960565, -4679.71422435, -467

In [95]:
w1 = rbm.w

In [106]:
w1

array([[ 4319.81245076,  4319.85858868,  4319.02174112,  4319.57171267,
         4319.61417782,  4319.41214906,  4319.71755049,  4319.77286019,
         4319.56533897,  4319.94963689,  4319.70020256,  4319.55662193],
       [-4910.9114493 , -4910.85971719, -4910.17555312, -4910.23925592,
        -4910.68279153, -4911.06868004, -4910.08110368, -4910.62925242,
        -4911.03069551, -4910.77722564, -4911.01608502, -4910.11582809],
       [-4577.98076833, -4578.79317856, -4577.94775523, -4578.3674941 ,
        -4578.80087916, -4577.7862863 , -4578.85879667, -4578.64234549,
        -4578.34777133, -4578.43424326, -4578.63813905, -4578.58065518],
       [-4590.86157741, -4591.24846363, -4589.66678938, -4590.74141121,
        -4590.80021768, -4590.54694093, -4590.85876845, -4591.19544586,
        -4591.21583903, -4591.20718531, -4591.48284782, -4591.30886154],
       [-4650.42541103, -4650.98186143, -4649.50828228, -4650.58956251,
        -4650.54867851, -4649.70681429, -4651.09609422, -465